In [ ]:
import numpy as np
from dask.distributed import Client, LocalCluster
import dask.dataframe as dd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
ddf = dd.read_parquet("D:\\uber-taxi-demand\\data\\yellow_tripdata_2022-01.parquet", npartitions=1)

In [ ]:
ddf.head().T

In [ ]:
ddf = ddf.set_index(ddf.tpep_pickup_datetime, sorted=True)

In [ ]:
ddf.head().T

In [ ]:
ddf.tail()

In [ ]:
ddf.isnull().sum().compute()

In [ ]:
ddf.ffill()

In [ ]:

ddf.resample('H').agg({'passenger_count': 'sum','fare_amount':'median', 'VendorID': 'count'})

In [ ]:
ddf.loc[ddf.tpep_pickup_datetime>'2021-12-31 23:00:00', ['passenger_count','fare_amount', 'VendorID']]

### optimize for memory

In [ ]:
###### Steps ######

In [ ]:
ddf = dd.read_parquet("D:\\uber-taxi-demand\\data\\yellow_tripdata_2022-01.parquet", npartitions=1)
s1 = ddf.loc[ddf.tpep_pickup_datetime>'2021-12-31 23:00:00', ['tpep_pickup_datetime','passenger_count','fare_amount', 'VendorID']]
s2 = s1.set_index('tpep_pickup_datetime', sorted=True)
s3 = s2.ffill()
s4 = s3.resample('H').agg({'passenger_count': 'sum', 'VendorID': 'count'})
s4.dask

In [ ]:
s4.compute().info()

In [ ]:
def optimeToFitMemory(ddf):
    new_types = dict(
        float32 = ['passenger_count',],
        int16 = ['VendorID',]
    )
    for key in new_types:
        for col in new_types[key]:
            ddf[col] = ddf[col].astype(key)
    return ddf

In [ ]:
s5 = s4.map_partitions(optimeToFitMemory)
s5.dask

In [ ]:
s5.compute().info()

In [ ]:
df = s5.compute().info()

In [ ]:
dsfdsf./

In [ ]:
cluster = LocalCluster(
    name='timeseries',
    n_workers=3,
    threads_per_worker=1,
    host='localhost',
    scheduler_port=8786,
    asynchronous=True
)
client = Client(cluster)

In [ ]:
client